<a href="https://colab.research.google.com/github/peteray-dev/GAN_Painter/blob/master/I_AM_SOMETHING_OF_A_PAINTER_MYSELF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>